In [1]:


import pandas as pd

from base.training import mse_weighted
from common import Predictor, DataStorage, L2Threshold
from common.model_utils import load_model_tflite
from simulation.model_simulator import ModelSimulator

if mse_weighted:
    print("mse_weighted loaded")

df_baseline = pd.read_pickle("zamg_vienna_hourly.pickle")

features = ["TL", "P", "RF", "SO"]
df_baseline = df_baseline[features]
df_baseline.dropna(inplace=True)

df_2019_2019 = df_baseline[df_baseline.index.year == 2019]

sim_data: pd.DataFrame = df_baseline.loc[df_baseline.index.year >= 2020]

THRESHOLD = 1.0
threshold = L2Threshold(THRESHOLD, [0], [0])

sim_data

mse_weighted loaded


,TL,P,RF,SO
time,,,,
2020-01-01 00:00:00,5.0,1012.4,60.0,0.0
2020-01-01 00:10:00,5.0,1012.5,60.0,0.0
2020-01-01 00:20:00,4.9,1012.5,61.0,0.0
2020-01-01 00:30:00,4.9,1012.5,61.0,0.0
2020-01-01 00:40:00,4.9,1012.6,60.0,0.0
...,...,...,...,...
2021-12-31 23:10:00,15.3,999.2,59.0,0.0
2021-12-31 23:20:00,15.2,999.3,59.0,0.0
2021-12-31 23:30:00,15.2,999.3,59.0,0.0


In [2]:
# Vienna 2019-2019 simple dense
model = load_model_tflite("models/zamg_vienna_2019_2019_simple_dense")
metadata = model.metadata
data_storage = DataStorage(metadata.input_features, metadata.output_features)
data_storage.add_measurement_df(df_2019_2019)
predictor = Predictor(model, data_storage)
predictor.update_prediction_horizon(sim_data.index.min())
model_simulator = ModelSimulator(predictor, threshold, sim_data)

model_simulator.run()

,TL,P,RF,SO,prediction,error,model
2020-01-01 00:00:00,5.0,1012.4,60.0,0.0,5.0,0.0,zamg_vienna_2019_2019_simple_dense
2020-01-01 00:10:00,5.0,1012.5,60.0,0.0,4.783937,0.216063,zamg_vienna_2019_2019_simple_dense
2020-01-01 00:20:00,4.9,1012.5,61.0,0.0,4.567873,0.332127,zamg_vienna_2019_2019_simple_dense
2020-01-01 00:30:00,4.9,1012.5,61.0,0.0,4.35181,0.54819,zamg_vienna_2019_2019_simple_dense
2020-01-01 00:40:00,4.9,1012.6,60.0,0.0,4.135746,0.764254,zamg_vienna_2019_2019_simple_dense
...,...,...,...,...,...,...,...
2021-12-31 23:10:00,15.3,999.2,59.0,0.0,14.906212,0.393788,zamg_vienna_2019_2019_simple_dense
2021-12-31 23:20:00,15.2,999.3,59.0,0.0,14.893139,0.306861,zamg_vienna_2019_2019_simple_dense
2021-12-31 23:30:00,15.2,999.3,59.0,0.0,14.880066,0.319934,zamg_vienna_2019_2019_simple_dense
2021-12-31 23:40:00,15.0,999.7,59.0,0.0,14.866993,0.133007,zamg_vienna_2019_2019_simple_dense


In [3]:
import numpy as np

results_2019_2019 = model_simulator.simulation_results

results_2019_2019['violation'] = results_2019_2019.apply(
    lambda row: threshold.is_threshold_violation(np.array([row['TL']]), np.array([row['prediction']])), axis=1
)
violations_2019_2019 = results_2019_2019[results_2019_2019['violation'] == True]
print(f"Violations / Measurements = {len(violations_2019_2019) / len(results_2019_2019)}")

Violations / Measurements = 0.06599597203222374


In [4]:
# Vienna 2010-2019 simple dense
model = load_model_tflite("models/zamg_vienna_2010_2019_simple_dense")
metadata = model.metadata
data_storage = DataStorage(metadata.input_features, metadata.output_features)
data_storage.add_measurement_df(df_2019_2019)
predictor = Predictor(model, data_storage)
predictor.update_prediction_horizon(sim_data.index.min())
model_simulator = ModelSimulator(predictor, threshold, sim_data)

model_simulator.run()

,TL,P,RF,SO,prediction,error,model
2020-01-01 00:00:00,5.0,1012.4,60.0,0.0,5.0,0.0,zamg_vienna_2010_2019_simple_dense
2020-01-01 00:10:00,5.0,1012.5,60.0,0.0,4.926071,0.073929,zamg_vienna_2010_2019_simple_dense
2020-01-01 00:20:00,4.9,1012.5,61.0,0.0,4.852142,0.047858,zamg_vienna_2010_2019_simple_dense
2020-01-01 00:30:00,4.9,1012.5,61.0,0.0,4.778214,0.121786,zamg_vienna_2010_2019_simple_dense
2020-01-01 00:40:00,4.9,1012.6,60.0,0.0,4.704285,0.195715,zamg_vienna_2010_2019_simple_dense
...,...,...,...,...,...,...,...
2021-12-31 23:10:00,15.3,999.2,59.0,0.0,15.141442,0.158558,zamg_vienna_2010_2019_simple_dense
2021-12-31 23:20:00,15.2,999.3,59.0,0.0,15.090111,0.109889,zamg_vienna_2010_2019_simple_dense
2021-12-31 23:30:00,15.2,999.3,59.0,0.0,15.038779,0.161221,zamg_vienna_2010_2019_simple_dense
2021-12-31 23:40:00,15.0,999.7,59.0,0.0,14.987447,0.012553,zamg_vienna_2010_2019_simple_dense


In [5]:
results_2010_2019 = model_simulator.simulation_results

results_2010_2019['violation'] = results_2010_2019.apply(
    lambda row: threshold.is_threshold_violation(np.array([row['TL']]), np.array([row['prediction']])), axis=1
)
violations_2010_2019 = results_2010_2019[results_2010_2019['violation'] == True]
print(f"Violations / Measurements = {len(violations_2010_2019) / len(results_2010_2019)}")

Violations / Measurements = 0.05489056087551299
